# Boilerplate

Package installation, loading, and dataloaders. There's also a simple model defined. You can change it your favourite architecture if you want.

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

# use_cuda = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print(device)
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



## NN defined as specified in the assignment. 3 fully connected layers (followed by ReLU activations)
## of size 50.
## interval analysis function also defined here.
class IntervalNet(nn.Module):
    def __init__(self):
        super(IntervalNet, self).__init__()

        # Define the network layers
        self.fc1 = nn.Linear(28*28, 50)  # First fully connected layer
        self.relu1 = nn.ReLU()                # First ReLU
        self.fc2 = nn.Linear(50, 50)          # Second fully connected layer
        self.relu2 = nn.ReLU()                # Second ReLU
        self.fc3 = nn.Linear(50, 50)          # Third fully connected layer
        self.relu3 = nn.ReLU()                # Third ReLU
        self.output_layer = nn.Linear(50, 10)  # Output layer

    def forward(self, x):
        # Forward pass through the layers
        # flatten the images
        x = x.view((-1, 28*28))

        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.output_layer(x)  # No activation at the output layer
        return x

    def interval_propagation(self, x_min, x_max):
        # Propagate through each layer of the model
        for layer in self.model.children():
            if isinstance(layer, nn.Linear):
                x_min, x_max = self._propagate_linear(layer, x_min, x_max)
            elif isinstance(layer, nn.ReLU):
                x_min, x_max = self._propagate_relu(x_min, x_max)
        return x_min, x_max

    def _propagate_linear(self, layer, x_min, x_max):
        W = layer.weight
        b = layer.bias

        # i think this is a fast way to do it instead of manually looping through the weights
        W_positive = torch.clamp(W, min=0)  # positive part of W
        W_negative = torch.clamp(W, max=0)  # negative part of W

        # Interval bounds propagation through linear layers
        y_min = W_positive @ x_min + W_negative @ x_max + b
        y_max = W_positive @ x_max + W_negative @ x_min + b
        return y_min, y_max

    def _propagate_relu(self, x_min, x_max):
        # Apply ReLU interval-wise
        return torch.relu(x_min), torch.relu(x_max)

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
# model = nn.Sequential(Normalize(), Net())
# image_size = 28 * 28
# classes = 10
model = IntervalNet()

model = model.to(device)
model.train()

loss_func = nn.CrossEntropyLoss()

cpu
Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 35755840.37it/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1190370.70it/s]


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9566461.25it/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7040106.71it/s]


Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



# Model Training

In [ ]:
def train_model(model, num_epochs):
    # TODO: implement this function that trains a given model on the MNIST dataset.
    # this is a general-purpose function for both standard training and adversarial training.
    # (toggle enable_defense parameter to switch between training schemes)
    optimizer = optim.SGD(model.parameters())

    model.train()

    for epoch in range(num_epochs):
        # code adapted from official pytorch guide here: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
        # running_loss = 0.0
        for inputs, labels in train_loader:
            # get the inputs; data is a list of [inputs, labels]
            # inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

    print("Training complete")


In [ ]:
def test_model(model):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    for test_inputs, true_labels in test_loader:
        test_inputs = test_inputs.to(device)
        true_labels = true_labels.to(device)

        # Forward pass
        output = model(test_inputs)
        test_loss += loss_func(output, true_labels).item()  # Sum up batch loss

        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability

        correct += pred.eq(true_labels.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")
    return accuracy

In [ ]:
train_model(model, 20)
torch.save(model.state_dict(), 'weights.pt')

Training complete


# Brief model evaluation

In [ ]:
test_model(model)

# Evaluate on L-infinity neighborhoods

In [ ]:
# Define L-infinity neighborhood sizes (evenly spaced between 0.01 and 0.1)
epsilon_values = torch.linspace(0.01, 0.1, steps=10)

def evaluate_robustness(test_loader, epsilon_values):
    model.eval()  # Set the model to evaluation mode

    for epsilon in epsilon_values:
        robust = 0
        total = 0

        for sample, t_label in test_loader:
            sample = sample.to(device)
            t_label = t_label.to(device)

            sample = sample.view((-1, 28*28))  # Flatten MNIST image
            total += 1

            # Define input intervals for L-infinity perturbation
            x_min = sample - epsilon  # Lower bound of the interval
            x_max = sample + epsilon  # Upper bound of the interval

            # Perform interval propagation
            output_min, output_max = model.interval_propagation(x_min, x_max)

            # Predicted label using the original (non-interval) input
            pred_original = model(sample).argmax(dim=1)

            # Check if the predicted class is consistent across the interval bounds
            pred_min = output_min.argmax(dim=1)
            pred_max = output_max.argmax(dim=1)

            # Evaluation in a neighborhood is robust, if the true label's minimum prediction
            # is larger than any other label's maximum
            true_class_min = output_min[t_label]
            output_max[t_label] = -1 # remove the true label's maximum from consideration
            if output_max.max() < true_class_min:
                robust += 1

        robustness = robust / total
        print(f"Robustness for epsilon {epsilon:.3f}: {robustness * 100:.2f}%")